# Securing The Box

This notebook helps users to secure their PYNQ instance for resilience in both testing and possible production scenarios. 

It is probably a no-brainer that `xilinx:xilinx` as a login is _really quite bad_, especially for systems that are going to potentially have to undergo security testing and assessment. As such, this notebook exists to help you set this part of the overall system up in order to fix this very low-hanging fruit of a bug. 

To do this, the script will do a couple of things:

1. It will change the default password from `xilinx` to a 24 character random string.
2. It will add an SSH key to the `xilinx` user's login, and allow you to download the public and private keys to use for both experimentation and/or production.

The concept for how much security you require is based on 'security level's which are described below. As a minimum, you should change the password and add an SSH key. For more advanced systems or those that may require more security, the script will also edit the SSH server configuration to prohibit password-based login, as well as prohibit `root` user access over SSH. These two actions effectively lock the system to only being accessible using the public/private key pair that has been generated - it is probably only for engineers who know what they are doing. 

As part of a good defence in depth strategy, every quantum computing system should have sufficient secuirty controls at every level. This script provides added security at the level of the controller, but your own threat modelling and risk management should be applied to the rest of your quantum computing system!

## Security Levels

The security levels in play can be summarised as follows:

* '0' - this is just changing the default password to something long and random, generating a key, and then adding that to authorized keys. 
* '1' - does everything at level 0, and then proceeds to edit the SSHD config file to prevent root login and prevent password login. 

**NB** - With security level 1 you MUST have downloaded the SSH private keyfile 'id_rsa' in order to successfully log back in! Run the cells at the end of this notebook to get download links.

The security hardening code is in the `secure-ssh.sh` file included in this folder. It is run in the following cell. 

## Setting the Security Level

The security level can be set in code as seen below, but also may be set using the BASH commend:

```
export QICK_SECURITY_LEVEL=0
```

## Running the code

When this code is run, the output from the script will be diplayed. 

**You MUST pay attention to the following output block if this is your first time running this script:**

```
========= WRITE THIS DOWN! =========

[!] New Password: ZkMY1QGcB7mNvBKnkpIKaE1z

===== THIS IS YOUR NEW PASSWORD!====
```

It is **IMPERATIVE** that you make note of the new password - it will no longer be 'xilinx'!! Note that your string may vary from this example. However, if you aren't running this script for the first time, this might fail, at which point you can ignore this password and keep your old one. 

In [4]:
import os
os.environ['QICK_SECURITY_LEVEL'] = "0"

!./secure-ssh.sh

0
tr: write error: Broken pipe
tr: write error


========= WRITE THIS DOWN! =========

[!] New Password: ZkMY1QGcB7mNvBKnkpIKaE1z

===== THIS IS YOUR NEW PASSWORD!====


[i] Changing 'xilinx' user password...
Changing password for mc.
Current password: passwd: Authentication token manipulation error
passwd: password unchanged
[i] Password Not Changed! Ignore the above string.
[i] Generating SSH keys. Private key is in the file 'id_rsa' and
    the public key is in 'id_rsa.pub'. You will need 'id_rsa'
    locally accessible to access the server with that key.
Generating public/private rsa key pair.
Your identification has been saved in ./id_rsa
Your public key has been saved in ./id_rsa.pub
The key fingerprint is:
SHA256:/pPJ9o8IoYWz1ADaU8fbG8TDftlMIUugd7IcZNUGMgs Qick server
The key's randomart image is:
+---[RSA 4096]----+
|    . ..Eo*o=o.. |
|   o o ..=== o+  |
|  . o . .==.o*   |
|     . +.o+=o o  |
|      + S o+     |
|     . * ..      |
|      o o. o     |
|         o*. .   |
|   

## The SSH Keys

In order to secure a production environment, the keys must be setup, and passwords must be disabled. Whilst this may introduce some complexity, it greatly reduces the potential attack surface and hardens the environment against attackers. 

To simplify this, the next cell, when run, will generate two download link buttons that will allow you to safely download the correct files. These can also be accessed from the file tree in the jupyter home page, or from the terminal.

In [5]:
# Soln inspired by: https://github.com/voila-dashboards/voila/issues/711 
from ipywidgets import Output, Button
from IPython.display import HTML, clear_output
from base64 import b64encode

download_output = Output()
display(download_output)
def make_download(text, filename, kind='text/json'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn1 = Button(description='Private Key File')
btn2 = Button(description='Public Key File')
def download_priv(e=None):
    make_download('./id_rsa', './id_rsa', kind='text/plain')
def download_pub(e=None):
    make_download('./id_rsa.pub', './id_rsa.pub', kind='text/plain')
btn1.on_click(download_priv)
btn2.on_click(download_pub)
print("You MUST download these files now!")
display(btn1)
display(btn2)

Output()

You MUST download these files now!


Button(description='Private Key File', style=ButtonStyle())

Button(description='Public Key File', style=ButtonStyle())